In [1]:
# Import required packages
import torch
import os
import numpy as np

from utils.solver import Solver, DEFAULT_SOLVER_PARAM_M7, DEFAULT_SOLVER_PARAM_M3
from utils.utils import load_numpy, save_numpy, nearest_neighbor_F

WorldModel::LoadRobot: /home/luca/.cache/jrl/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tris)
LoadAssimp: Loaded model /ho

In [3]:
solver = Solver(solver_param=DEFAULT_SOLVER_PARAM_M7)
solver.path_following(load_time='1111215818', num_traj=100, shrink_ratio=0, enable_evaluation=True, enable_plot=False)

Model load successfully from /home/luca/ikpflow/weights/panda/1107-1013.pth
F load successfully from /home/luca/ikpflow/data/panda/train/F-2400000-7-7-1.npy
knn load successfully from /home/luca/ikpflow/data/panda/train/knn-2400000-7-7-1-normFalse.pickle
/home/luca/ikpflow/data/panda/trajectory/1111215818/ load successfully.
            l2_err      ang_err         mjac
count  2000.000000  2000.000000  2000.000000
mean      0.034725     1.058400    98.907798
std       0.017095     0.353795   126.754300
min       0.001242     0.004200    13.845312
25%       0.020368     0.912784    15.149041
50%       0.037628     1.176879    24.579557
75%       0.048461     1.354860   260.291557
max       0.096360     1.499386   345.718811
avg_inference_time: 0.002


In [ ]:
import numpy as np
import pandas as pd
from time import time
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

In [ ]:
set_seed()

# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot

P = solver.sample_P_path(load_time='1111215818')

In [ ]:
num_traj = 100
num_steps = P.shape[0]

Qs = np.empty((num_traj, num_steps, solver._robot.n_dofs))
l2_err_arr = np.empty((num_traj, num_steps))
ang_err_arr = np.empty((num_traj, num_steps))
mjac_arr = np.empty((num_traj, num_steps))
runtime_arr = np.empty((num_traj))

In [ ]:
begin = time()
for i in range(num_traj):
    sol, l2_err_arr[i], ang_err_arr[i], _, _, runtime = ik_solver.solve_n_poses(
        P, 
        latent_scale=.01,
        refine_solutions=False, 
        return_detailed=True)
    # runtime_arr[i] = runtime
    mjac_arr[i] = solver.max_joint_angle_change(sol)
    Qs[i] = sol.detach().cpu().numpy()

avg_runtime = (time() - begin) / num_traj

In [ ]:
df = pd.DataFrame({'l2_err': l2_err_arr.flatten(), 'ang_err': ang_err_arr.flatten(), 'mjac': mjac_arr.flatten()})
print(df.describe())
print(f"avg_inference_time: {avg_runtime}")